# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 23 08:43:29 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660394,35.3,1454491,77.7,1454491,77.7
Vcells,1226581,9.4,8388608,64.0,1975136,15.1


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 500041

PARAM$experimento <- 6300
PARAM$dataset <- "gerencial_competencia_2025.csv"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

In [7]:
head(dataset)

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,ccallcenter_transacciones,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29186441,202005,1,61,282,2354.94,43761.59,1682.14,-139.02,712.60,⋯,0,20,47,0,6507,0.00,0,7671,1043.97,CONTINUA
29187961,202005,1,61,312,5066.31,47268.33,3875.62,-3536.03,4551.11,⋯,0,228,232,0,5939,2017.56,0,8090,7495.47,CONTINUA
29193101,202005,1,67,365,4809.42,37996.66,6982.73,-2962.24,1109.46,⋯,0,133,149,0,2026,2838.66,0,2026,1653.93,CONTINUA
29193281,202005,1,54,158,2979.52,88436.59,3350.01,-908.55,592.70,⋯,2,10,115,0,3125,844.56,0,3350,2029.29,CONTINUA
29198891,202005,1,54,312,3008.70,22567.32,273.68,-643.87,2975.52,⋯,0,124,32,0,7202,234.60,0,7242,1395.87,CONTINUA
29205441,202005,1,72,6,394.19,1607.93,159.49,-9.92,210.01,⋯,0,41,32,0,157,0.00,0,157,58.65,CONTINUA


In [8]:
# Ejecuta un pequeño script JavaScript desde R para mantener activa la sesión
IRdisplay::display_javascript("
function clickConnect() {
  console.log('Manteniendo sesión activa desde R...');
  const btn = document.querySelector('#top-toolbar > colab-connect-button');
  if (btn && btn.shadowRoot) {
    const connect = btn.shadowRoot.querySelector('#connect');
    if (connect) connect.click();
  }
}
setInterval(clickConnect, 60000);
")

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [9]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [11]:
#PASO 1
library(readxl)
library(data.table)
library(dplyr)
fapce <- read_excel("/content/datasets/Indice-FACPCE.xlsx") %>%
  mutate(
    # ---- FOTO_MES CORREGIDO ----
    # Eliminar el último caracter: "2020050" → "202005"
    foto_mes = gsub("[^0-9]", "", foto_mes),
    foto_mes = substr(foto_mes, 1, 6),

    # ---- IPC ----
    IPC = gsub("[^0-9]", "", IPC),
    IPC = as.numeric(IPC),

    # ---- coeficiente ajustes ----
    coeficiente_de_ajuste = gsub(",", ".", coeficiente_de_ajuste),
    coeficiente_de_ajuste = gsub("[^0-9.]", "", coeficiente_de_ajuste),
    coeficiente_de_ajuste = as.numeric(coeficiente_de_ajuste)
  ) %>%
  as.data.table()





In [12]:
head(fapce)

foto_mes,IPC,coeficiente_de_ajuste
<chr>,<dbl>,<dbl>
202005,3149,1.678311
202006,322,1.641304
202007,3282,1.610299
202008,3371,1.567784
202009,3466,1.524812
202010,3597,1.469280


In [13]:
#PASO 2
dataset <- as.data.table(dataset)

# asegurar foto_mes como texto tipo "202005"
dataset[, foto_mes := as.character(foto_mes)]
dataset[, foto_mes := gsub("[^0-9]", "", foto_mes)]
dataset[, foto_mes := substr(foto_mes, 1, 6)]





In [14]:
#PASO 3
cols_borrar <- c("coeficiente_de_ajuste", "coeficiente_de_ajuste.x", "coeficiente_de_ajuste.y")

for (col in cols_borrar) {
  if (col %in% names(dataset)) dataset[, (col) := NULL]
}





In [15]:
#PASO 4
dataset <- merge(
  dataset,
  fapce[, .(foto_mes, coeficiente_de_ajuste)],
  by = "foto_mes",
  all.x = TRUE
)



In [16]:
#PASO 5 MERGE POR FOTO_MES
vars_ajustar <- c(
  "mrentabilidad",
  "mrentabilidad_annual",
  "mcomisiones",
  "mactivos_margen",
  "mpasivos_margen",
  "mcuenta_corriente",
  "mcaja_ahorro",
  "mcuentas_saldo",
  "mtarjeta_visa_consumo",
  "mtarjeta_master_consumo",
  "mprestamos_personales",
  "mpayroll",
  "Master_mpagominimo",
  "Visa_mpagominimo"
)

dataset[, (vars_ajustar) := lapply(.SD, as.numeric), .SDcols = vars_ajustar]



In [17]:
#PASO 6
dataset[
  ,
  (vars_ajustar) := lapply(.SD, function(x) x * coeficiente_de_ajuste),
  .SDcols = vars_ajustar
]



In [18]:
head(dataset)

foto_mes,numero_de_cliente,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria,coeficiente_de_ajuste
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>
202005,29186441,1,61,282,3952.3207,73445.539,2823.1534,-233.31874,1195.9641,⋯,20,47,0,6507,0.0000,0,7671,1752.10589,CONTINUA,1.678311
202005,29187961,1,61,312,8502.8416,79330.938,6504.4940,-5934.55654,7638.1760,⋯,228,232,0,5939,3386.0923,0,8090,12579.72656,CONTINUA,1.678311
202005,29193101,1,67,365,8071.7004,63770.196,11719.1896,-4971.55872,1862.0185,⋯,133,149,0,2026,4764.1531,0,2026,2775.80821,CONTINUA,1.678311
202005,29193281,1,54,158,5000.5599,148424.064,5622.3572,-1524.82907,994.7347,⋯,10,115,0,3125,1417.4340,0,3350,3405.77887,CONTINUA,1.678311
202005,29198891,1,54,312,5049.5330,37874.972,459.3200,-1080.61383,4993.8467,⋯,124,32,0,7202,393.7317,0,7242,2342.70338,CONTINUA,1.678311
202005,29205441,1,72,6,661.5732,2698.606,267.6738,-16.64884,352.4620,⋯,41,32,0,157,0.0000,0,157,98.43292,CONTINUA,1.678311


In [19]:
# sin codigo en esta primera version del workflow

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [20]:
# Función auxiliar (ya correcta)
atributos_presentes <- function(patributos) {
  atributos <- unique(patributos)
  comun <- intersect(atributos, colnames(dataset))
  return(length(atributos) == length(comun))
}

# --- Necesario 1: foto_mes debe ser numérico PARA LA OPERACIÓN ---
if (atributos_presentes(c("foto_mes"))) {
  dataset[, kmes := as.integer(foto_mes) %% 100]
}

# --- Necesario 2: asegurar tipos numéricos para el cálculo ---
if (atributos_presentes(c("mpayroll", "cliente_edad"))) {
  dataset[, mpayroll := as.numeric(mpayroll)]
  dataset[, cliente_edad := as.numeric(cliente_edad)]
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]
}


In [21]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "foto_mes"                    "numero_de_cliente"          
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "coeficiente_de_ajuste"       "kmes"                       
[35] "mpayroll_sobre_edad"

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [22]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [23]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [24]:
ncol(dataset)
colnames(dataset)

[1] 163

[1] "foto_mes"                           "numero_de_cliente"                 
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "coeficiente_de_ajuste"              "kmes"                              
 [35] "mpayroll_sobre_edad"                "internet_lag1"                     
 [37] "cliente_edad_lag1"                  "cliente_antiguedad_lag1"           
 [39] "mrentabilidad_lag1"                 "mrentabilidad_annual_lag1"         
 [41] "mcomisiones_lag1"                   "mactivos_margen_lag1"              
 [43] "mpasivos_margen_lag1"               "cproductos_lag1"                   
 [45] "mcuenta_corriente_lag1"             "mcaja_ahorro_lag1"                 
 [47] "cdescubierto_preacordado_lag1"      "mcuentas_saldo_lag1"               
 [49] "ctarjeta_visa_transacciones_lag1"   "mtarjeta_visa_consumo_lag1"        
 [51] "mtarjeta_master_consumo_lag1"       "mprestamos_personales_lag1"        
 [53] "cpayroll_trx_lag1"                  "mpayroll_lag1"                     
 [55] "ccomisiones_mantenimiento_lag1"     "ccallcenter_transacciones_lag1"    
 [57] "chomebanking_transacciones_lag1"    "ctrx_quarter_lag1"                 
 [59] "Master_status_lag1"                 "Master_fechaalta_lag1"             
 [61] "Master_mpagominimo_lag1"            "Visa_status_lag1"                  
 [63] "Visa_fechaalta_lag1"                "Visa_mpagominimo_lag1"             
 [65] "coeficiente_de_ajuste_lag1"         "kmes_lag1"                         
 [67] "mpayroll_sobre_edad_lag1"           "internet_lag2"                     
 [69] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [71] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [73] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [75] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [77] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [79] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
 [81] "ctarjeta_visa_transacciones_lag2"   "mtarjeta_visa_consumo_lag2"        
 [83] "mtarjeta_master_consumo_lag2"       "mprestamos_personales_lag2"        
 [85] "cpayroll_trx_lag2"                  "mpayroll_lag2"                     
 [87] "ccomisiones_mantenimiento_lag2"     "ccallcenter_transacciones_lag2"    
 [89] "chomebanking_transacciones_lag2"    "ctrx_quarter_lag2"                 
 [91] "Master_status_lag2"                 "Master_fechaalta_lag2"             
 [93] "Master_mpagominimo_lag2"            "Visa_status_lag2"                  
 [95] "Visa_fechaalta_lag2"                "Visa_mpagominimo_lag2"             
 [97] "coeficiente_de_ajuste_lag2"         "kmes_lag2"                         
 [99] "mpayroll_sobre_edad_lag2"           "internet_delta1"                   
[1

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [25]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [26]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0

PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")



In [27]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [28]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [29]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



In [30]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [31]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘parallelMap’


Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [48]:
#--------------------------------------
#  HIPERPARÁMETROS OPTIMIZADOS
#  (respetando valores neutros fijos)
#--------------------------------------

PARAM$hipeparametertuning$num_interations <- 80   # grid amplio

PARAM$lgbm$param_fijos <- list(
  objective              = "binary",
  metric                 = "auc",
  first_metric_only      = TRUE,
  boost_from_average     = TRUE,
  feature_pre_filter     = FALSE,
  verbosity              = -100,
  force_row_wise         = TRUE,
  seed                   = PARAM$semilla_primigenia,

  # 🔒 PARÁMETROS NEUTRALES (NO TOCAR)
  max_bin                = 31,
  feature_fraction       = 0.5,
  learning_rate          = 0.03,

  # robustez
  bagging_fraction       = 0.7,
  bagging_freq           = 1,

  # entrenamiento largo, define early stopping
  num_iterations         = 15000,
  early_stopping_rounds  = 400
)


#--------------------------------------
# BUSQUEDA DE HIPERPARÁMETROS
#--------------------------------------

PARAM$hipeparametertuning$hs <- makeParamSet(

  # tamaño del árbol
  makeIntegerParam("num_leaves", lower = 16L, upper = 256L),

  # tamaño mínimo de hojas (mayor rango = mayor descubrimiento)
  makeIntegerParam("min_data_in_leaf", lower = 50L, upper = 8192L),

  # control de complejidad
  makeIntegerParam("max_depth", lower = -1L, upper = 16L),

  # regularización avanzada para evitar sobreajuste
  makeNumericParam("lambda_l1", lower = 0, upper = 5),
  makeNumericParam("lambda_l2", lower = 0, upper = 5),

  # evita splits insignificantes
  makeNumericParam("min_gain_to_split", lower = 0, upper = 0.1)
)


Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [49]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [50]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [51]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sun Nov 23 11:30:10 PM 2025 AUC 0.934916898180866

Sun Nov 23 11:33:44 PM 2025 AUC 0.928492118476571

Sun Nov 23 11:37:06 PM 2025 AUC 0.928157069736943

Sun Nov 23 11:39:16 PM 2025 AUC 0.929368898857451

Sun Nov 23 11:43:27 PM 2025 AUC 0.924977814027096

Sun Nov 23 11:50:18 PM 2025 AUC 0.93232664239919

Sun Nov 23 11:52:54 PM 2025 AUC 0.926369216296437

Sun Nov 23 11:56:26 PM 2025 AUC 0.928028935606227

Sun Nov 23 11:59:16 PM 2025 AUC 0.928996313537035

Mon Nov 24 12:03:12 AM 2025 AUC 0.926523069936211

Mon Nov 24 12:06:02 AM 2025 AUC 0.917796741732105

Mon Nov 24 12:10:27 AM 2025 AUC 0.927651252725457

Mon Nov 24 12:14:48 AM 2025 AUC 0.922723070399626

Mon Nov 24 12:17:59 AM 2025 AUC 0.931355325444473

Mon Nov 24 12:20:09 AM 2025 AUC 0.92775644783458

Mon Nov 24 12:22:58 AM 2025 AUC 0.929542911031352

Mon Nov 24 12:27:51 AM 2025 AUC 0.934998690853818

Mon Nov 24 12:32:01 AM 2025 AUC 0.927356289346794

Mon Nov 24 12:35:30 AM 2025 AUC 0.9

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [52]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf max_depth lambda_l1 lambda_l2 min_gain_to_split
        <int>            <int>     <int>     <num>     <num>             <num>
1:        253               53        -1    2.2522  1.639357         0.0214429
   num_iterations
            <int>
1:            400


### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [53]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 240562

##### Final Training Hyperparameters

In [54]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [55]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [56]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [57]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [58]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [59]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [60]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [61]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Warning message in dir.create("kaggle"):
“'kaggle' already exists”


Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 


In [62]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [63]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Nov 24 05:50:57 AM 2025"